## Run only once!

This first cell only needs to be run once to install the WOS Lite SDK

In [ ]:
# If the woslite_client to be located even though you installed it at the command line
# try this suggestion from
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
# You only need to do it once ever unless you reinstall Jupyter notebooks.

# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install git+https://github.com/Clarivate-SAR/woslite_py_client.git

## Configuration

Import modules, define functions, set configuration variables

In [1]:
# Import modules
from __future__ import print_function
# The WOS Lite client must be built directly from GitHub using
# pip3 install git+https://github.com/Clarivate-SAR/woslite_py_client.git
import woslite_client
from woslite_client.rest import ApiException
from pprint import pprint # not really necessary, regular printing is fine
from pathlib import Path
import time

# Configuration variables

# Database to search. Must be a valid database ID, one of the following: 
# BCI/BIOABS/BIOSIS/CCC/DCI/DIIDW/MEDLINE/WOK/WOS/ZOOREC. WOK represents all databases.
database_id = 'WOS'  # probably only valid value for API key, only authorized for WOS Lite
lang = 'en'  # str | Language of search. This element can take only one value: en for English. If no language is specified, English is passed by default. (optional)

# This function will load some credential from a text file, either in the home directory or current working directory
# The value of the directory variable should be either 'home' or 'working'
# Keeping the credential in the home directory prevents accidentally uploading it with the notebook.
# The function returns a single string, so if there is more than one credential (e.g. key plus secret), additional
# parsing of the return value may be required. 
def load_credential(filename, directory):
    cred = ''
    # to change the script to look for the credential in the working directory, change the value of home to empty string
    if directory == 'home':
        home = str(Path.home()) #gets path to home directory; works for both Win and Mac
        credential_path = home + '/' + filename
    else:
        directory = 'working'
        credential_path = filename
    try:
        with open(credential_path, 'rt', encoding='utf-8') as file_object:
            cred = file_object.read()
    except:
        print(filename + ' file not found - is it in your ' + directory + ' directory?')
        exit()
    return(cred)


## Instantiate API objects

For testing, this cell only needs to be run once each time the notebook is opened. The instances can be reused for multiple API calls in later cells

In [2]:
# Load the API key from plain text file
# Key is a single random hex string on one line in the file with no trailing newline
clarivate_api_key = load_credential('clarivate_api_key.txt', 'home')

# Configure API key authorization: key
configuration = woslite_client.Configuration()
configuration.api_key['X-ApiKey'] = clarivate_api_key

# create an instance of the API class
integration_api_instance = woslite_client.IntegrationApi(woslite_client.ApiClient(configuration))
search_api_instance = woslite_client.SearchApi(woslite_client.ApiClient(configuration))

print('done')

done


## Set search values

These values are the same regardless of search type

In [3]:
# count. Number of records returned in the request
# For testing, use 1. Max is 100.
count = 10  # integer

# first_record. Specific record, if any within the result set to return.
# Cannot be less than 1 and greater than 100000.
first_record = 11  # integer 

# Order by field(s). Field name and order by clause separated by '+', 
# use A for ASC and D for DESC, ex: PY+D.
# Multiple values are separated by comma. (optional)
sort_field = 'PY+D'  


## Search by query

This search uses the `search_api_instance` to retrieve records based on search results.

```
AD=Address
AI=Author Identifiers (ResearcherID and ORCID) 
AU=Author 
CF=Conference
CI=City
CU=Country
DO=DOI
ED=Editor
FG=Grant Number 
FO=Funding Agency 
FT=Funding Text 
GP=Group Author 
IS=ISSN/ISBN
OG=Organization - Enhanced 
OO=Organization 
PMID=PubMed ID 
PS=Province/State
PY=Year Published 
SA=Street Address 
SG=Suborganization 
SO=Publication Name 
SU=Research Area 
TI=Title
TS=Topic
UT=Accession Number 
WC=Web of Science Category 
ZP=Zip/Postal Code

Example: TS=particle swarm AND PY=(2007 OR 2008)
```

The API details are at their [Swagger site](https://api.dev-stable.clarivate.com/swagger-ui/?url=https%3A%2F%2Fdeveloper.dev-stable.clarivate.com%2Fapis%2Fwoslite%2Fswagger)

In [4]:
# usr_query. User query for requesting data, ex: TS=(cadmium). 
# The query parser will return errors for invalid queries.

#usr_query = 'OG=(Vanderbilt University OR Vanderbilt University Peabody College) AND AI=(0000-0003-0328-0792)'
#usr_query = 'AU=(Baskauf CJ)'
#usr_query = 'AI=(0000-0003-4365-3135)'
usr_query = 'OG=(Vanderbilt University OR Vanderbilt University Peabody College)'

try:
    # Find record(s) by user query
    api_response = search_api_instance.root_get(database_id, 
                                                usr_query, 
                                                count, 
                                                first_record, 
                                                lang=lang,
                                                #publish_time_span='2020-12-01+2020-12-31',
                                                load_time_span='4M',
                                                sort_field=sort_field)
    # for more details look at the models
    #firstAuthor = api_response.data[0].author.authors[0]
    print("Response: ")
    pprint(api_response)
    #pprint("First author: " + firstAuthor)
except ApiException as e:
    print("Exception when calling SearchApi->root_get: %s\\n" % e)


Response: 
{'data': [{'author': {'authors': ['Manz, Kevin M.',
                                  'Becker, Jennifer C.',
                                  'Grueter, Carrie A.',
                                  'Grueter, Brad A.'],
                      'book_authors': None,
                      'book_group_authors': None},
           'doctype': {'doctype': ['Article']},
           'keyword': {'keywords': None},
           'other': {'contributor_researcher_id_names': ['Grueter, Brad'],
                     'contributor_researcher_id_researcher_i_ds': None,
                     'identifier_article_no': None,
                     'identifier_doi': ['10.1016/j.biopsych.2020.07.023'],
                     'identifier_eissn': ['1873-2402'],
                     'identifier_ids': ['QI2EB'],
                     'identifier_isbn': None,
                     'identifier_issn': ['0006-3223'],
                     'identifier_xref_doi': None,
                     'researcher_id_disclaimer': ['Re

In [28]:
articles_list = []
for record in api_response.data:
    if record.other.identifier_doi:
        article_dict = {}
        article_dict['authors'] = record.author.authors
        article_dict['doi'] = record.other.identifier_doi[0]
        article_dict['title'] = record.title.title[0]
        article_dict['type'] = record.doctype.doctype[0]
        print(article_dict)

{'authors': ['Manz, Kevin M.', 'Becker, Jennifer C.', 'Grueter, Carrie A.', 'Grueter, Brad A.'], 'doi': '10.1016/j.biopsych.2020.07.023', 'title': 'Histamine H-3 Receptor Function Biases Excitatory Gain in the Nucleus Accumbens', 'type': 'Article'}
{'authors': ['Garrabrants, Andrew C.', 'Kosson, David S.', 'Brown, Kevin G.', 'Jr., Daniel P. Fagnant', 'Helms, Gregory', 'Thorneloe, Susan A.'], 'doi': '10.1016/j.jhazmat.2020.124635', 'title': 'Methodology for scenario-based assessments and demonstration of treatment effectiveness using the Leaching Environmental Assessment Framework (LEAF)', 'type': 'Article'}
{'authors': ['Kim, Ji-Woon', 'Shin, Chan Young'], 'doi': '10.1111/apha.13643', 'title': 'Deciphering the role of T-type calcium channels in regulating adult hippocampal neurogenesis', 'type': 'Editorial Material'}
{'authors': ['Varlotto, John M.', 'Sun, Zhuoxin', 'Ky, Bonnie', 'Upshaw, Jenica', 'Katz, Sharyn, I', 'Fitzgerald, Thomas J.', 'Wakelee, Heather', 'Diehn, Maximilian', 'Man

## Item search by ID

This search uses the `integration_api_instance` to retrieve a record for a specific item or list of items separated by commas.

In [ ]:
# unique_id. Primary item(s) id to be searched, ex: WOS:000270372400005. 
# Cannot be null or an empty string. Multiple values are separated by comma.
unique_id = 'WOS:000270372400005'

try:
    # Find record(s) by specific id
    api_response = integration_api_instance.id_unique_id_get(database_id, unique_id, count, first_record, lang=lang,
                                                             sort_field=sort_field)
    # for more details look at the models
    firstAuthor = api_response.data[0].author.authors[0]
    print("Response: ")
    pprint(api_response)
    pprint("First author: " + firstAuthor)
except ApiException as e:
    print("Exception when calling IntegrationApi->id_unique_id_get: %s\\n" % e)
